## Plotting accelerometer data vs mount motions

Craig Lage - Apr 7, 2022

In [ ]:
import sys, time, datetime
import numpy as np
from scipy.signal import butter,filtfilt, savgol_filter
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from astropy.time import Time, TimeDelta
import astropy.units as u
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [ ]:
# Get EFD client and the butler
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
# Get one header data using Gen3 butler
# This confirms that the DATE_BEG and DATE_END timestamps remain in TAI, as specified.
expId = 2022040600457
mData_457 = butler.get('raw.metadata', detector=0, exposure=expId)
expId = 2022040600458
mData_458 = butler.get('raw.metadata', detector=0, exposure=expId)

In [ ]:
# Need to convert DATE_BEG and DATE_END to UTC to sync up with the EFD
date_beg_457 = Time(mData_457['DATE-BEG'], format='isot', scale='tai')
date_end_457 = Time(mData_457['DATE-END'], format='isot', scale='tai')
date_beg_458 = Time(mData_458['DATE-BEG'], format='isot', scale='tai')
date_end_458 = Time(mData_458['DATE-END'], format='isot', scale='tai')

In [ ]:
filename = '20220406T223707Z.pkl'
# Unpickle the accel dataframe
file = open(f'/scratch/labJackData/{filename}', 'rb')
df = pkl.load(file)
file.close()

In [ ]:
buffer_time = 0.0
buffer = TimeDelta(buffer_time, format='sec')
start = date_beg_457.utc - buffer
end = date_end_458.utc + buffer
tstart = start.value
tend = end.value
print(tstart, tend)
# Mask out bad regions
df = df.where(abs(df['ELM2']) < 10.0) 
print(f"Approximately {len(df) / 200:.2f} seconds of data")
# Select a subset of the data if desired
# and apply a rolling average
rolling = 1 # number of data points to average
timeSubset = True
if timeSubset:
    subdf = df.loc[tstart:tend]
    subdf = subdf.rolling(rolling).sum() / rolling
print(f"Approximately {len(subdf) / 200:.2f} seconds of data")

tstart = Time(tstart)
tend = Time(tend)
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  tstart, tend)
el = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "elevationCalculatedAngle",  tstart, tend)
doOffset = True
if doOffset:
    offset = (tstart.jd - az.index[0].to_julian_date()) * 86400.0
    az.index += pd.DateOffset(seconds=offset)
    el.index += pd.DateOffset(seconds=offset)
# Calculate the tracking errors
az_vals = np.array(az.values.tolist())[:,0]
el_vals = np.array(el.values.tolist())[:,0]
times = np.array(az.values.tolist())[:,1]
times = times - times [0]

# Fit with a quadratic
az_fit = np.polyfit(times, az_vals, 4)
el_fit = np.polyfit(times, el_vals, 4)

az_model = az_fit[0] * times * times * times * times + az_fit[1] * times * times * times \
    + az_fit[2] * times *times + az_fit[3] * times + az_fit[4]
el_model = el_fit[0] * times * times * times * times + el_fit[1] * times * times * times \
    + el_fit[2] * times * times + el_fit[3] * times + el_fit[4]

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600


fig = plt.figure(figsize = (8, 10))
plt.subplots_adjust(hspace=1.0, wspace=1.5)

plt.suptitle(f"Mount Tracking vs Accel 20220406", fontsize = 18)
plt.subplots_adjust(wspace=1.0, hspace=1.0)
plt.subplot(4,3,1)
name = "Az-Error"
plt.title(name, fontsize=12)
plt.plot(times, az_error)
#az['azimuthCalculatedAngle'].plot()
plt.ylabel('ArcSec')
plt.subplot(4,3,2)
name = "El-Error"
plt.title(name, fontsize=12)
plt.plot(times, el_error)
#el['elevationCalculatedAngle'].plot()
plt.ylabel('Arcsec')
plotcounter = 4
for accel in ['M2', 'T', 'M1']:
    for axis in ['AZ', 'EL', 'Z']:
        name = axis + accel
        plt.subplot(4,3,plotcounter)
        plt.title(name, fontsize=12)
        data = subdf[name].to_list()
        filtered_data = butter_lowpass_filter(data, 20.0, 200, 2)
        plt.plot(f
        plt.ylabel('Acceleration(g)')
        plotcounter += 1

plt.savefig(f"/scratch/labJackData/EarthQuake_20220406.pdf")
        

In [ ]:
plt.close()

In [ ]:
azm2_data = subdf['AZM2'].to_list()

In [ ]:
azm2_data[0:3]

In [ ]:
azm2_filtered = butter_lowpass_filter(subdf['AZM2'].to_list(), 20.0, 200, 2)

In [ ]:
plt.plot(azm2_filtered)

In [ ]:
azm2_savgol = savgol_filter(subdf['AZM2'].to_list(), 20, 5)

In [ ]:
plt.plot(azm2_savgol)